## Simulating melts of polymer chains

In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import gsd.hoomd
from hoomd.md import nlist

import unyt as u
import glob
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
import os

import flowermd
from flowermd.utils import get_target_box_mass_density
from flowermd.base import Simulation, Molecule
from flowermd.library import FF_from_file
from flowermd.base.system import Pack

from polymer_dictionary import polymer_dictionary

# Rerun with the original seed first

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/pkg_resources/__init__.py:3146: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)



Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



In [2]:
# Updates atom names to fit with hoomd conventions
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

## Single molecule sim:

In [3]:
key_list = sorted(list(polymer_dictionary.keys())) # Aligning dictionary and path to aid automation
path = os.getcwd() # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [4]:
mol2_path = os.getcwd() + "/mol2/"
xml_path = os.getcwd() + "/xml/"

In [4]:
system_file = mol2_path + "alternate_PCPDTPT_nC16.mol2"
ff_filepath = xml_path + "alternate_PCPDTPT_nC16.xml"

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.


In [5]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=1, log_write_freq=1, device=cpu, gsd_file_name="alternate_PCPDTPT_nC16_melt.gsd",log_file_name="alternate_PCPDTPT_nC16_melt.txt")

Initializing simulation state from a gsd.hoomd.Frame.


In [6]:
sim.run_NVT(n_steps=1, kT=5.0, tau_kt=1.0)
sim.flush_writers()

In [24]:
sim.reference_values

{'energy': unyt_quantity(1.046, 'kJ/mol'),
 'length': unyt_quantity(0.35635949, 'nm'),
 'mass': unyt_quantity(32.06, 'amu')}

## Running sims of every polymer in the Danielsen et al. paper

In [18]:
pack_seeds = 

for i in range(len(key_list)):
    system_file = mol2_path + "10_mers/" + key_list[i] + "_10mer.mol2"
    ff_filepath = xml_path + key_list[i] + ".xml"
    gsd_path = os.getcwd() + "/gsd_files/10_mers/"
    num_mers = "_10mer" # The number of monomers being simulated is determined by the mol2, this is for naming the resulting gsd and .txt files
    espmol = espaloma_mol(system_file)
    molecule = Molecule(num_mols=1, compound=espmol)
    cpu = hoomd.device.CPU()
    
    molff = FF_from_file(ff_filepath)
    system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
    system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
    system.hoomd_snapshot
    hoomd_forces = system.hoomd_forcefield
    hoomd_forces
    lj_force = hoomd_forces[3]
    
    sim = Simulation.from_system(system=system, gsd_write_freq=10000, log_write_freq=10000, device=cpu, gsd_file_name=gsd_path+key_list[i]+num_mers+".gsd",log_file_name=gsd_path+key_list[i]+num_mers+".txt")
    sim.run_NVT(n_steps=1e6, kT=5.0, tau_kt=1.0)
    sim.flush_writers()

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 10000 of 1000000; TPS: 1089.39; ETA: 15.1 minutes
Step 20000 of 1000000; TPS: 1014.77; ETA: 16.1 minutes
Step 30000 of 1000000; TPS: 995.07; ETA: 16.2 minutes
Step 40000 of 1000000; TPS: 1009.04; ETA: 15.9 minutes
Step 50000 of 1000000; TPS: 994.19; ETA: 15.9 minutes
Step 60000 of 1000000; TPS: 984.8; ETA: 15.9 minutes
Step 70000 of 1000000; TPS: 1007.12; ETA: 15.4 minutes
Step 80000 of 1000000; TPS: 990.38; ETA: 15.5 minutes
Step 90000 of 1000000; TPS: 983.29; ETA: 15.4 minutes
Step 100000 of 1000000; TPS: 998.6; ETA: 15.0 minutes


IOStream.flush timed out


Step 110000 of 1000000; TPS: 994.68; ETA: 14.9 minutes
Step 120000 of 1000000; TPS: 988.62; ETA: 14.8 minutes
Step 130000 of 1000000; TPS: 999.5; ETA: 14.5 minutes
Step 140000 of 1000000; TPS: 994.13; ETA: 14.4 minutes


IOStream.flush timed out
IOStream.flush timed out


...

Step 470000 of 1000000; TPS: 977.94; ETA: 9.0 minutes
Step 480000 of 1000000; TPS: 976.24; ETA: 8.9 minutes
Step 490000 of 1000000; TPS: 974.5; ETA: 8.7 minutes
Step 500000 of 1000000; TPS: 974.85; ETA: 8.5 minutes
Step 510000 of 1000000; TPS: 970.48; ETA: 8.4 minutes
Step 520000 of 1000000; TPS: 966.21; ETA: 8.3 minutes
Step 530000 of 1000000; TPS: 966.1; ETA: 8.1 minutes
Step 540000 of 1000000; TPS: 965.39; ETA: 7.9 minutes
Step 550000 of 1000000; TPS: 967.61; ETA: 7.8 minutes
Step 560000 of 1000000; TPS: 966.02; ETA: 7.6 minutes
Step 570000 of 1000000; TPS: 965.04; ETA: 7.4 minutes
Step 580000 of 1000000; TPS: 967.23; ETA: 7.2 minutes
Step 590000 of 1000000; TPS: 966.08; ETA: 7.1 minutes
Step 600000 of 1000000; TPS: 964.29; ETA: 6.9 minutes
Step 610000 of 1000000; TPS: 966.09; ETA: 6.7 minutes
Step 620000 of 1000000; TPS: 964.07; ETA: 6.6 minutes
Step 630000 of 1000000; TPS: 962.17; ETA: 6.4 minutes
Step 640000 of 1000000; TPS: 963.56; ETA: 6.2 minutes
Step 650000 of 1000000; TPS: 9

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 10000 of 1000000; TPS: 481.0; ETA: 34.3 minutes
Step 20000 of 1000000; TPS: 541.7; ETA: 30.2 minutes
Step 30000 of 1000000; TPS: 538.17; ETA: 30.0 minutes
Step 40000 of 1000000; TPS: 546.46; ETA: 29.3 minutes
Step 50000 of 1000000; TPS: 551.01; ETA: 28.7 minutes
Step 60000 of 1000000; TPS: 549.26; ETA: 28.5 minutes
Step 70000 of 1000000; TPS: 559.64; ETA: 27.7 minutes
Step 80000 of 1000000; TPS: 557.21; ETA: 27.5 minutes
Step 90000 of 1000000; TPS: 560.14; ETA: 27.1 minutes
Step 100000 of 1000000; TPS: 560.21; ETA: 26.8 minutes
Step 110000 of 1000000; TPS: 553.42; ETA: 26.8 minutes
Step 120000 of 1000000; TPS: 556.45; ETA: 26.4 minutes
Step 130000 of 1000000; TPS: 554.71; ETA: 26.1 minutes
Step 140000 of 1000000; TPS: 558.14; ETA: 25.7 minutes
Step 150000 of 1000000; TPS: 560.01; ETA: 25.3 minutes
Step 160000 of 1000000; TPS: 555.4; ETA: 25.2 minutes
Step 170000 of 1000000; TP

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 10000 of 1000000; TPS: 336.03; ETA: 49.1 minutes
Step 20000 of 1000000; TPS: 387.49; ETA: 42.2 minutes
Step 30000 of 1000000; TPS: 400.98; ETA: 40.3 minutes
Step 40000 of 1000000; TPS: 416.3; ETA: 38.4 minutes
Step 50000 of 1000000; TPS: 426.73; ETA: 37.1 minutes
Step 60000 of 1000000; TPS: 433.07; ETA: 36.2 minutes
Step 70000 of 1000000; TPS: 431.24; ETA: 35.9 minutes
Step 80000 of 1000000; TPS: 434.69; ETA: 35.3 minutes
Step 90000 of 1000000; TPS: 436.7; ETA: 34.7 minutes
Step 100000 of 1000000; TPS: 437.04; ETA: 34.3 minutes
Step 110000 of 1000000; TPS: 434.12; ETA: 34.2 minutes
Step 120000 of 1000000; TPS: 435.34; ETA: 33.7 minutes
Step 130000 of 1000000; TPS: 437.46; ETA: 33.1 minutes
Step 140000 of 1000000; TPS: 438.67; ETA: 32.7 minutes
Step 150000 of 1000000; TPS: 436.34; ETA: 32.5 minutes
Step 160000 of 1000000; TPS: 436.29; ETA: 32.1 minutes
Step 170000 of 1000000; T